## Notebook for Production of DE1 tables

#### Import packages & functions

In [ ]:
library(haven)
library(PxWebApiData)
suppressPackageStartupMessages(library(tidyverse))
library(klassR)

library(httr)
library(jsonlite)

library(glue)

In [ ]:
source("Functions.r")

#### Statbank data import

In [ ]:
# import data

df_DE1 <- ApiData(07459,
              Alder = list("agg:FemAarigGruppering",
                           c("F00-04", "F05-09", "F10-14",
                             "F15-19", "F20-24", "F25-29",
                             "F30-34", "F35-39", "F40-44",
                             "F45-49", "F50-54", "F55-59",
                             "F60-64", "F65-69", "F70-74",
                             "F75-79", "F80-84", "F85-89",
                             "F90-94", "F95-99", "F100G5+")),
              Region = c("0301", "1103", "4601", "5001", "3005"),
              Tid = "2022")[[2]]

In [ ]:
# import data for median

df_DE1073V <- ApiData(13536,
                      Region = list("vs:Kommune",
                                    c("3005", "0301", "1103", "4601", "5001")),
                      ContentsCode = "Medianalder",
                      Tid = "2022")[[2]]

#### Data processing
1. Add a column with municipality names onto the dataframe based on region code (klass = 131)
2. Load a correspondence table between city name and city code
3. Merge the dataframe and city correspondence 

In [ ]:
# import klass correspondence for city names

df_DE1$name <- ApplyKlass(df_DE1$Region, klass = 131)
df_DE1073V$name <- ApplyKlass(df_DE1073V$Region, klass = 131)

city_correspondence <- GetKlass(550) %>%
                    select(code, name)

In [ ]:
# merge klass correspondence with dataframe

df_DE1 <- merge.data.frame(df_DE1, city_correspondence,
                           by = "name")

df_DE1073V <- merge.data.frame(df_DE1073V, city_correspondence,
                               by = "name")

#### Summary tables

In [ ]:
# DE1001V - DE1003V

DE1001V_2022 <- df_DE1 %>%
                   group_by(code) %>%
                   summarise(Value = sum(value))

DE1002V_2022 <- sum_rows(Kjonn, "1")

DE1003V_2022 <- sum_rows(Kjonn, "2")

In [ ]:
# DE1073V

DE10073_2022 <- df_DE1073V %>%
                    select(code, value) %>%
                    rename(Value = value)

In [ ]:
# DE1137V - DE1139V

DE1137V_2022 <- df_DE1 %>%
                    filter(Alder %in% c("F85-89",
                                        "F90-94",
                                        "F95-99",
                                        "F100G5+")) %>%
                    group_by(code) %>%
                    summarise(Value = sum(value))

DE1137V_2022 <- df_DE1 %>%
                    filter(Alder %in% c("F85-89",
                                        "F90-94",
                                        "F95-99",
                                        "F100G5+")) %>%
                    group_by(code) %>%
                    summarise(Value = sum(value))

In [ ]:
DE1137V_2022

In [ ]:
var_list <- list("DE1040V", "DE1041V", "DE1042V", # 0-4 years
                      "DE1046V", "DE1047V", "DE1048V", # 15-19 years
                      "DE1049V", "DE1050V", "DE1051V", # 20-24 years
                      "DE1074V", "DE1075V", "DE1076V", # 5-9 years
                      "DE1077V", "DE1078V", "DE1079V", # 10-14 years
                      "DE1101V", "DE1102V", "DE1103V", # 25-29 years
                      "DE1104V", "DE1105V", "DE1106V", # 30-34 years
                      "DE1107V", "DE1108V", "DE1109V", # 35-39 years
                      "DE1110V", "DE1111V", "DE1112V", # 40-44 years
                      "DE1113V", "DE1114V", "DE1115V", # 45-49 years
                      "DE1116V", "DE1117V", "DE1118V", # 50-54 years
                      "DE1119V", "DE1120V", "DE1121V", # 55-59 years
                      "DE1122V", "DE1123V", "DE1124V", # 60-64 years
                      "DE1125V", "DE1126V", "DE1127V", # 65-69 years
                      "DE1128V", "DE1129V", "DE1130V", # 70-74 years
                      "DE1131V", "DE1132V", "DE1133V", # 75-79 years
                      "DE1134V", "DE1135V", "DE1136V") # 80-84 years

In [ ]:
var_age <- list("F00-04", "F00-04", "F00-04",
                     "F15-19", "F15-19", "F15-19",
                     "F20-24", "F20-24", "F20-24",
                     "F05-09", "F05-09", "F05-09",
                     "F10-14", "F10-14", "F10-14",
                     "F25-29", "F25-29", "F25-29",
                     "F30-34", "F30-34", "F30-34",
                     "F35-39", "F35-39", "F35-39",
                     "F40-44", "F40-44", "F40-44",
                     "F45-49", "F45-49", "F45-49",
                     "F50-54", "F50-54", "F50-54",
                     "F55-59", "F55-59", "F55-59",
                     "F60-64", "F60-64", "F60-64",
                     "F65-69", "F65-69", "F65-69",
                     "F70-74", "F70-74", "F70-74",
                     "F75-79", "F75-79", "F75-79",
                     "F80-84", "F80-84", "F80-84")

In [ ]:
var_gender <- list("total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2",
                   "total", "1", "2")

In [ ]:
variable_correspondence <- data.frame(cbind(var_list, var_age, var_gender))

# export this to outside notebook

In [ ]:
variable_correspondence

In [ ]:
for (i in seq(1, nrow(variable_correspondence), 3)) {
    assign(
        paste(as.character(variable_correspondence$var_list[i]),
              "2022",
              sep = "_"),
        sum_rows(Alder, as.character(variable_correspondence$var_age[i])))
}

In [ ]:
table_formatting(DE1077V_2022)

#### Gender separated tables
Note, for the gender seperated tables we do not have to sum over rows, but simply filter at the age group and gender. This is done using the function: filter_rows, and specifying age_group as the first variable and the gender code as the second.

In [ ]:
index <- setdiff(1:nrow(variable_correspondence),
                 seq(1, nrow(variable_correspondence), 3))

In [ ]:
for (i in index) {
    assign(
        paste(as.character(variable_correspondence$var_list[i]),
              "2022",
              sep = "_"),
        filter_rows(as.character(variable_correspondence$var_age[i]),
                    as.character(variable_correspondence$var_gender[i])))
}

## Table formatting

In [ ]:
table_formatting(DE1001V_2022)